In [2]:
%cd /content/drive/My Drive/NLP_project
from glob import glob
import pandas as pd
import random
from define import *

/content/drive/My Drive/NLP_project


# Crawler Data

In [0]:
%cd /content/drive/My Drive/NLP_project/data_crawler/

/content/drive/My Drive/NLP_project


In [0]:
!pip install scrapy

In [0]:
!scrapy startproject data_crawler

/bin/bash: scrapy: command not found


In [0]:
!scrapy crawl Url -o url_VNexpress.csv

In [0]:
!scrapy crawl VNexpress -o data_VNexpress.csv

# Read Data

Split Data to Train,Test,Val

In [0]:
paths = glob('data_*.csv')

train = pd.DataFrame()
val = pd.DataFrame()
test = pd.DataFrame()

for path in paths:
    df = pd.read_csv(path)
    df = df.dropna()
    df = df.where(df.description != '\n').dropna()
    df = df.sample(frac=1).reset_index(drop=True)
    df['description'] = [title.strip('(Dân trí) -') for title in df['description']]
    cates = []
    for cate in df['category']:
        if cate == 'giáo dục - khuyến học':
            cates.append('giáo dục')
        else:
            cates.append(cate.lower())
    df['category'] = cates
    num = df.count()['title']
    idx_train = 8*num//10
    idx_val = 9*num//10
    df_train = df.iloc[0:idx_train]
    df_val = df.iloc[idx_train:idx_val]
    df_test = df.iloc[idx_val:-1]
    train = train.append(df_train)
    val = val.append(df_val)
    test = test.append(df_test)
train = train.sample(frac=1).reset_index(drop=True)
val = val.sample(frac=1).reset_index(drop=True)
test = test.sample(frac=1).reset_index(drop=True)

Save Data

In [0]:
train.to_csv(DATA_TRAIN_CSV, index=False)
val.to_csv(DATA_VAL_CSV, index=False)
test.to_csv(DATA_TEST_CSV, index=False)

# Pre-Processing Data

In [0]:
!pip install pyvi

In [3]:
!python preprocessing.py

Read Data
Read Data
Read Data


# Feature Extraction

In [0]:
!python VectorizationWord.py

2020-06-14 17:10:43.584711: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Building TF-IDF features
Building TF-IDF
tcmalloc: large alloc 4333830144 bytes == 0xa176000 @  0x7febceb37001 0x7febcb5aa765 0x7febcb60ebb0 0x7febcb610a4f 0x7febcb6a7048 0x50a635 0x50cd96 0x509758 0x50a48d 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50bfb4 0x509758 0x50a48d 0x50bfb4 0x509758 0x50a48d 0x50bfb4 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00
Building TF-IDF features
Building TF-IDF features
tcmalloc: large alloc 4333830144 bytes == 0x14cfcc000 @  0x7febceb351e7 0x59203c 0x7febcb69dedd 0x5667d2 0x5a4be1 0x7febcb69eb88 0x5667d2 0x5a4be1 0x4e24ce 0x4e3386 0x45441a 0x50a3ea 0x50bfb4 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7febce732b97 0x5b250a
Traceback (most recent call last):
  File "VectorizationWord.py", line 130, in <module>
    pickle.dump(features_train, 

# Draft

In [0]:
sentences = []
for text in train:
    words = text['content'].split(' ')
    sentences.append(words)

In [0]:
from gensim.models import Word2Vec
model_skipgram = Word2Vec(sentences=sentences,
                workers = 4,
                min_count = 2,
                size = 300,
                sg = 1,
                window = 10)

model_cbow = Word2Vec(sentences=sentences,
                workers = 4,
                min_count = 2,
                size = 300,
                sg = 0,
                window = 10)

In [0]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
tokenizer = Tokenizer(
    num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^`{|}~\t\n', lower=True,
    split=' ', char_level=False, oov_token=None, document_count=0
)

In [0]:
texts = train.content
tokenizer.fit_on_texts(texts)
sequences_train = tokenizer.texts_to_sequences(texts)
X_train = pad_sequences(sequences_train)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
train.head()

,content,category
0,ngày không lây_nhiễm ncov cộng_đồng,sức khỏe
1,h ngày bộ y_tế không ghi_nhận thêm ca nhiễm nc...,sức khỏe
2,hội_thảo giáo_dục phổ_thông môn tin_học,giáo dục
3,chương_trình do đại_học trực_tuyến funix và hệ...,giáo dục
4,phương_nam mở_rộng thị_trường tấm cách_nhiệt p...,kinh doanh


In [0]:
tfidf = TfidfVectorizer(use_idf=True,
                                 smooth_idf=True,
                                 sublinear_tf=False,
                                 min_df=0.0, max_df=1.0)
tfidf.fit(train['content'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=0.0, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [0]:
train['content'][0]

'ngày không lây_nhiễm ncov cộng_đồng'

In [0]:
temp = tfidf.transform([train['content'][0]])

In [0]:
temp[0, 11393]

0.5670797566542594

In [0]:
tfidf.vocabulary_['lây_nhiễm']

11393

In [0]:
from VectorizationWord import FeatureExtraction
import numpy as np

In [0]:
train = pd.read_csv(PROCESSED_DATA_TRAIN_CSV)
val = pd.read_csv(PROCESSED_DATA_VAL_CSV)
test = pd.read_csv(PROCESSED_DATA_TEST_CSV)

features_train = FeatureExtraction(data=train['content'], train = True).get_features_tfidf()
features_val = FeatureExtraction(data=val['content']).get_features_tfidf()
features_test = FeatureExtraction(data=test['content']).get_features_tfidf()

# pickle.dump(features_train, open(FEATURES_TRAIN_TFIDF.replace(".p",".p"), "wb"))
# pickle.dump(features_test, open(FEATURES_TEST_TFIDF.replace(".p",".p"), "wb"))
# pickle.dump(features_val, open(FEATURES_VAL_TFIDF.replace(".p",".p"), "wb"))


Building TF-IDF features
Building TF-IDF features
Building TF-IDF features


OverflowError: ignored

In [0]:
pickle.dump(train['category'], open(LABELS_TRAIN, "wb"))
pickle.dump(test['category'], open(LABELS_TEST, "wb"))
pickle.dump(val['category'], open(LABELS_VAL, "wb"))
